In [ ]:
import pandas as pd

data = pd.read_csv('/home/jvdzwaan/data/tmp/ocr/align_metadata.csv', index_col=2, encoding='utf-8')
data

In [ ]:
import codecs

with codecs.open('/home/jvdzwaan/Downloads/ocr/BAObi1-ds.ocr.txt', encoding='utf-8') as f:
    ocr = f.read()
with codecs.open('/home/jvdzwaan/Downloads/ocr/BAObi1-ds.gs.txt', encoding='utf-8') as f:
    gs = f.read()
print ocr
print len(ocr)
print '---'
print gs
print len(gs)

In [ ]:
cigar = data['cigar'][0]
print cigar

In [ ]:
import re
from collections import Counter
matches = re.findall(r'(\d+)(.)', cigar)
offset1 = 0
offset2 = 0

gs_a = []
ocr_a = []

seq1 = ocr
seq2 = gs

empty_char = ''

for m in matches:
    n = int(m[0])
    typ = m[1]

    if typ == '=':
        # sanity check - strings should be equal
        assert(seq1[offset1:offset1+n] == seq2[offset2:offset2+n])

        for c in seq1[offset1:offset1+n]:
            ocr_a.append(c)
        for c in seq2[offset2:offset2+n]:
            gs_a.append(c)
        
        offset1 += n
        offset2 += n
    elif typ == 'D':  # Inserted
        for _ in range(n):
            ocr_a.append(empty_char)
        for c in seq2[offset2:offset2+n]:
            gs_a.append(c)

        offset2 += n
    elif typ == 'X':
        for c in seq1[offset1:offset1+n]:
            ocr_a.append(c)
        for c in seq2[offset2:offset2+n]:
            gs_a.append(c)

        offset1 += n
        offset2 += n
    elif typ == 'I':  # Deleted
        for c in seq1[offset1:offset1+n]:
            ocr_a.append(c)
        for _ in range(n):
            gs_a.append(empty_char)

        offset1 += n

In [ ]:
print len(ocr_a), len(gs_a)

In [ ]:
for i, j in zip(ocr_a, gs_a):
    print i, j

In [ ]:
raw_text = ' '.join([''.join(ocr_a), ''.join(gs_a)])

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))    

n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

In [ ]:
print char_to_int

In [ ]:
import numpy as np

def create_data(ocr_texts, gs_texts, seq_length=25):
    dataX = []
    dataY = []
    for ocr, gs in zip(ocr_texts, gs_texts):
        text_length = len(ocr)
        for i in range(0, text_length-seq_length +1, 1):
            seq_in = ocr[i:i+seq_length]
            seq_out = gs[i:i+seq_length]
            dataX.append(''.join(seq_in))
            dataY.append(''.join(seq_out))
            print dataX[-1], dataY[-1]
        
    X = np.zeros((len(dataX), seq_length, n_vocab), dtype=np.bool)
    Y = np.zeros((len(dataY), seq_length, n_vocab), dtype=np.bool)

    #for i, sentence in enumerate(dataX):
        #print sentence
    #    for j, c in enumerate(sentence):
    #        X[i, j, char_to_int[c]] = 1
        #print X[i]
        #print X[i].shape

    #for i, sentence in enumerate(dataY):
        #print sentence
    #    for j, c in enumerate(sentence):
    #        Y[i, j, char_to_int[c]] = 1
        #print Y[i]
        #print Y[i].shape
    return X, Y

X, Y = create_data([ocr_a], [gs_a])

In [ ]:
print X.shape